In [ ]:
def read_data():
    train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv',nrows=100000)
    specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv',nrows=100000)
    sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv',nrows=100000)
    train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv',nrows=100000)
    test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv',nrows=100000)
    return train, test, train_labels, specs, sample_submission

def encode_title(train, test, train_labels):
    train['title_event_code'] = train['title'].astype(str)+train['event_code'].astype(str)
    test['title_event_code'] = test['title'].astype(str)+test['event_code'].astype(str)
    all_title_event_code = list(set(train["title_event_code"]).union(set(test["title_event_code"])))
    list_of_user_activities = list(set(train['title']).union(set(test['title'])))
    list_of_event_code = list(set(train['event_code']).union(set(test['event_code'])))
    list_of_event_id = list(set(train['event_id']).union(set(test['event_id'])))
    list_of_worlds = list(set(train['world']).union(set(test['world'])))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    return train, test, train_labels, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code


train, test, train_labels, specs, sample_submission = read_data()
train, test, train_labels, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code=encode_title(train,test,train_labels)
win_code={col:4100 for col in ['Mushroom Sorter (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)',
       'Chest Sorter (Assessment)']}
win_code['Bird Measurer (Assessment)']=4110
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    compiled_val=[]
    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort = False)), total = 17000):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort = False), total = 1000):
        test_data,val_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
        compiled_val+=(val_data)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    reduce_val = pd.DataFrame(compiled_val)
    return reduce_train, reduce_test,reduce_val

from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
%matplotlib inline
# import catboost as cat
import time
from collections import Counter
import datetime
# from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15
from collections import defaultdict
import lightgbm as lgb
import xgboost as xgb

def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    all_assessments=[]
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
    
    
    
    ass_try_non_rec=0
    ass_true_non_rec=0
    ass_false_non_rec=0


    ass_try_rec=0
    ass_true_rec=0
    ass_false_rec=0
  
  
    try_non_rec=0
    true_non_rec=0
    false_non_rec=0

    cnt=0
    try_rec=0
    true_rec=0
    false_rec=0
    cum_acc=0
    for i, session in user_sample.groupby('game_session', sort=False):
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        
        rec=session.loc[session['event_code'].isin([4100,4110])]
        non_rec=session.loc[~(session['event_code'].isin([4100,4110]))]
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            features={}
            all_attempts = session.loc[session['event_code']==win_code[session_title]]
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features['true_rec_ass']=ass_true_rec
            features['false_rec_ass']=ass_false_rec
            features['tot_rec_ass']=ass_try_rec
    
      
            features['true_non_rec_ass']=ass_true_non_rec
            features['false_non_rec_ass']=ass_false_non_rec
            features['tot_non_rec_ass']=ass_try_non_rec
    
      
            features['true_rec_non_ass']=true_rec
            features['false_rec_non_ass']=false_rec
            features['tot_rec_non_ass']=try_rec
    
      
            features['true_non_rec']=true_non_rec
            features['false_non_rec']=false_non_rec
            features['tot_non_rec']=try_non_rec
            if cnt!=0:
                features['accuracy_cum']=cum_acc/cnt
            else:
                features['accuracy_cum']=0
            cnt+=1
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session['title'].iloc[0]
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            cum_acc+=accuracy
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            true=rec.event_data.str.contains('true').sum()
            false=rec.event_data.str.contains('false').sum()
            tot=true+false
        
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                
                ass_true_rec+=true
                ass_false_rec+=false
                ass_try_rec=ass_false_rec+ass_true_rec
                
                
                all_assessments.append(features)
      
            else:
                ass_true_non_rec+=non_rec.event_data.str.contains('true').sum()
                ass_false_non_rec+=non_rec.event_data.str.contains('false').sum()
                ass_tot_non_rec=ass_true_non_rec+ass_false_non_rec
      
    
    
            
        
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')
        true_non_rec+=(non_rec['event_data'].str.contains('true')).sum()
        false_non_rec+=(non_rec['event_data'].str.contains('false')).sum()
        tot_non_rec=true_non_rec+false_non_rec
        true_rec+=(rec['event_data'].str.contains('true')).sum()
        false_rec+=(rec['event_data'].str.contains('false')).sum()
        tot_rec=false_rec+true_rec
    if test_set:
        return all_assessments[-1],all_assessments[:-1]
    return all_assessments
reduce_train, reduce_test, reduce_val= get_train_and_test(train, test)
for df in [reduce_train,reduce_test,reduce_val]:
    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
import lightgbm as lgb
trn_ft=lgb.Dataset(reduce_train.select_dtypes(exclude=object).drop(['accuracy_group'],1),reduce_train['accuracy_group'])
val_ft=lgb.Dataset(reduce_val.select_dtypes(exclude=object).drop(['accuracy_group'],1),reduce_val['accuracy_group'])
mod=lgb.train( {'feature_fraction': 0.7975023410800904,'n_estimators':7297,'learning_rate':0.001,
  'max_depth': 14,
  'subsample': 0.7531064105216918},trn_ft)
pre=mod.predict(reduce_test.select_dtypes(exclude=object).drop(['accuracy_group'],1))
x,y,z=1.21519937, 1.70523477, 2.25945307
res=[]
for i in pre:
        if i<x:
            res.append(0)
        elif i<y:
            res.append(1)
        elif i<z:
            res.append(2)
        else:
            res.append(3)


sample_submission['accuracy_group']=res
sample_submission.to_csv('submission.csv', index=False)
